In [149]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'


In [150]:
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0
def convert_to_float(x):
    """
    Функция для конвертирования строки в float. Если ошибка то возващаем 0
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом


    try:
        return float(x)
    except ValueError:
        return 0

In [151]:
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

In [152]:
def prepare_column_to_int(cell):
    """
    Функция для обработки ячеек в инт, чтобы они в экселе потом не отображались как строки
    """
    try:
        float_value = float(cell)
        int_value = int(float_value)
        return int_value
    except ValueError:
        return cell

In [153]:
region = 0

In [154]:
file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/2022-11-27_64_Реестр УПП с дополнительными колонками.xlsx'

# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Заголовок Реестр УПП от 25.10.xlsx'

In [155]:
# Создаем список колонок которые нужно загрузить
use_cols = list(range(25))


# Загружаем датафреймы
df_upp = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols,dtype={4:str,5:str})
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32],dtype={7:str,8:str,9:str})

# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_upp.columns = list(map(str,list(df_upp.columns)))
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_upp.columns = list(map(lambda x:x.strip(),list(df_upp.columns)))
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))




# Фильтруем датафрейм отбирая только те записи в которых есть 1 в графе 30
transfer_df=df_table_3[df_table_3['30'] == 1]

# перемещаем площадь выдела
transfer_df.insert(7,'12',transfer_df['33'])

# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=['30','33'],inplace=True)

transfer_df['17'] =transfer_df['17'].astype(str) # Приводим колонку к строковому формату


transfer_df['17'] =transfer_df['17'].apply(lambda x:x.replace(' ',''))
# Заменяем категории таблицы 3 на категории Реестра УПП
transfer_df['17'] =transfer_df['17'].replace(regex={'120':'2','100':'1',r'130|131|132|133|134|135|136':'3',
                                                    r'141|142|143|144|145|146|147|148|149|150|151|152':'4','nan':'0'})

transfer_df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

In [156]:
# Бурятия
if region == 0:
    
    # Создаем датафрейм для проверки наличия записи в реестре
    checked_df = df_upp[['1','2','3','4','5']]
    checked_df = checked_df.astype(str) # делаем данные строковыми
    
    # Готовим 4 и 5 колонки чтобы они были разделеныпри склеивании
    checked_df['4'] = checked_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    checked_df['5'] = checked_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    
    
    # Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
    checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
    # Удаляем пробелы
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace('nan',''))


    # делаем строковыми первые 5 колонок
    transfer_df[['1','2','3','4','5',]] = transfer_df[['1','2','3','4','5',]].astype(str)
    
     # Готовим 4 и 5 колонки чтобы они были разделеныпри склеивании
    transfer_df['4'] = transfer_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    transfer_df['5'] = transfer_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    
    transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace('nan',''))

    # Мержим по полю айди
    merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)


    # Отбираем только те значения которые есть в правом датафрейме
    added_df = merged_df[merged_df['_merge']=='right_only']

    added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','ID','_merge'],inplace=True) # удаляем лишние колонки

    added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5'},inplace=True) # переименовываем колонки для корректного добавления

    itog_df = pd.concat([df_upp,added_df],ignore_index=True)


    # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
    itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
    itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
    itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)

    # Заменяем nan и пробелы в итоговом файле
    # приводим к строковому виду
    itog_df[['1','2','3']] = itog_df[['1','2','3']].astype(str)
    # Заменяем
    itog_df[['1','2','3']] = itog_df[['1','2','3']].apply(lambda x:x.replace(' ',''))

    itog_df[['1','2','3','4','5']] = itog_df[['1','2','3','4','5']].apply(lambda x:x.replace('nan',np.nan))
 

   
    # Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    itog_df['4'] = itog_df['4'].astype(str)
    itog_df['5'] = itog_df['5'].astype(str)
    
    # Очищаем от разделителей
    itog_df['4'] = itog_df['4'].apply(lambda x:x.replace('кв.',''))
    itog_df['5'] = itog_df['5'].apply(lambda x:x.replace('в.',''))
    
    # конвертируем в инт чтобы корректно отображалось
    itog_df['4'] = itog_df['4'].apply(prepare_column_to_int)
    itog_df['5'] = itog_df['5'].apply(prepare_column_to_int)
   
   
    # Сортируем
    itog_df.sort_values(by=['1','2','3','4','5'],inplace=True)
    
  

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = load_workbook(header_reestr) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(itog_df,index=False,header=False):
        wb['Реестр УПП'].append(row)


    #Заменяем nan и пробелы в таблице с добавленными участками
    # приводим к строковому виду
    added_df[['1','2','3']] = added_df[['1','2','3']].astype(str)
    added_df[['1','2','3']] = added_df[['1','2','3']].apply(lambda x:x.replace(' ',''))
    added_df[['1','2','3']] = added_df[['1','2','3']].apply(lambda x:x.replace('nan',''))
    

        # Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    added_df['4'] = added_df['4'].astype(str)
    added_df['5'] = added_df['5'].astype(str)
    
        # Очищаем от разделителей
    added_df['4'] = added_df['4'].apply(lambda x:x.replace('кв.',''))
    added_df['5'] = added_df['5'].apply(lambda x:x.replace('в.',''))
    
        # конвертируем в инт чтобы корректно отображалось
    added_df['4'] = added_df['4'].apply(prepare_column_to_int)
    added_df['5'] = added_df['5'].apply(prepare_column_to_int)

    

    # Переименовываем после соединения колонки в таблице с добавленными участками
    added_df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                            },inplace=True)

    # Сохраняем файл с добавляемыми данными, чтобы пользователи знали что именно добавилось
    added_df.to_excel(f'{path_to_end_folder}/Бурятия Участки из таблицы 3 добавленные в реестр УПП от {current_time}.xlsx',index=False)

    wb.save(f'{path_to_end_folder}/Бурятия Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')
    


    # Якутия
elif region == 1:        
    # Создаем датафрейм для проверки наличия записи в реестре
    checked_df = df_upp[['1','2','3','4','5','11']]
    checked_df = checked_df.astype(str) # делаем данные строковыми
    
    checked_df['4'] = checked_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    checked_df['5'] = checked_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    checked_df['11'] = checked_df['11'].apply(lambda x:'г.' + x) # Добавляем разделитель год
    
        
    # Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
    checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5','11'] ].sum(axis=1)
        
    # Удаляем пробелы
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))
    checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace('nan',''))

    # делаем строковыми первые 6 колонок в таблице 3
    transfer_df[['1','2','3','4','5','11']] = transfer_df[['1','2','3','4','5','11']].astype(str)
    
             # Готовим 4 и 5 11 колонки чтобы они были разделеныпри склеивании
    transfer_df['4'] = transfer_df['4'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    transfer_df['5'] = transfer_df['5'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    transfer_df['11'] = transfer_df['11'].apply(lambda x:'г.' + x) # Добавляем разделитель год
    
    transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5','11'] ].sum(axis=1)
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))
    transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace('nan',''))

    # Мержим по полю айди
    merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)

    # Отбираем только те значения которые есть в правом датафрейме
    added_df = merged_df[merged_df['_merge']=='right_only']

    added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','11_x','ID','_merge'],inplace=True) # удаляем лишние колонки

    added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5','11_y':'11'},inplace=True) # переименовываем колонки для корректно

    itog_df = pd.concat([df_upp,added_df],ignore_index=True)

    # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
    itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
    itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
    itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)

    # Заменяем nan и пробелы в итоговом файле
    # приводим к строковому виду
    itog_df[['1','2','3']] = itog_df[['1','2','3']].astype(str)
    # Заменяем
    itog_df[['1','2','3']] = itog_df[['1','2','3']].apply(lambda x:x.replace(' ',''))
    itog_df[['1','2','3','4','5','11']] = itog_df[['1','2','3','4','5','11']].apply(lambda x:x.replace('nan',np.nan))


        # Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    itog_df['4'] = itog_df['4'].astype(str)
    itog_df['5'] = itog_df['5'].astype(str)
    itog_df['11'] = itog_df['11'].astype(str)
    
    # Очищаем от разделителей
    itog_df['4'] = itog_df['4'].apply(lambda x:x.replace('кв.',''))
    itog_df['5'] = itog_df['5'].apply(lambda x:x.replace('в.',''))
    itog_df['11'] = itog_df['11'].apply(lambda x:x.replace('г.',''))
    
        # конвертируем в инт чтобы корректно отображалось
    itog_df['4'] = itog_df['4'].apply(prepare_column_to_int)
    itog_df['5'] = itog_df['5'].apply(prepare_column_to_int)
    itog_df['11'] = itog_df['11'].apply(prepare_column_to_int)
   
   
    # Сортируем
    itog_df.sort_values(by=['1','2','3','4','5','11'],inplace=True)
    
  


    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    wb = load_workbook(header_reestr) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(itog_df,index=False,header=False):
        wb['Реестр УПП'].append(row)


    #Заменяем nan и пробелы в таблице с добавленными участками
    # приводим к строковому виду
    added_df[['1','2','3','11']] = added_df[['1','2','3','11']].astype(str)
    added_df[['1','2','3','11']] = added_df[['1','2','3','11']].apply(lambda x:x.replace(' ',''))
    added_df[['1','2','3','11']] = added_df[['1','2','3','11']].apply(lambda x:x.replace('nan',''))

     #   Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    added_df['4'] = added_df['4'].astype(str)
    added_df['5'] = added_df['5'].astype(str)
    added_df['11'] = added_df['11'].astype(str)
    
    # Очищаем от разделителей
    added_df['4'] = added_df['4'].apply(lambda x:x.replace('кв.',''))
    added_df['5'] = added_df['5'].apply(lambda x:x.replace('в.',''))
    added_df['11'] = added_df['11'].apply(lambda x:x.replace('г.',''))
    
        
    # конвертируем в инт год лесоустройства
            # конвертируем в инт чтобы корректно отображалось
    added_df['4'] = added_df['4'].apply(prepare_column_to_int)
    added_df['5'] = added_df['5'].apply(prepare_column_to_int)
    added_df['11'] = added_df['11'].apply(prepare_column_to_int)
    
     # Сортируем
    added_df.sort_values(by=['1','2','3','4','5','11'],inplace=True)
    
    

  # Переименовываем после соединения колонки в таблице с добавленными участками
    added_df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',},inplace=True)

    # Сохраняем файл с добавляемыми данными, чтобы пользователи знали что именно добавилось
    added_df.to_excel(f'{path_to_end_folder}/Саха(Якутия) Участки из таблицы 3 добавленные в реестр УПП от {current_time}.xlsx',index=False)

    wb.save(f'{path_to_end_folder}/Саха(Якутия) Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')

        
        